# Programação Orientada a Objetos - Monkey Patching e Consulta de Atributos
A orientação a objetos é um paradigma de programação conhecido por seus quatro pilares principais: Encapsulamento; Abstração; Herença e Polimorfismo. O propósito deste paradigma é solucionar os problemas tradicionais da programação procedural, onde muitas vezes há pouco aproveitamento de código e fraca associação entre dados e às funções reponsáveis por manipular estes dados. Neste estudo, buscamos revisar os conceitos fundamentais da orientação a objetos e identificar como eles são implementados na linguagem Python.

## Monkey Patching
O monkey patching é a ideia de ser possível adicionar comportamento para um objeto existente em tempo de execução. Isto é útil pois evita ter que implementar todos protocolos para uma classe, deixando os casos raros para serem feitos apenas quando for necessário. Por Python ser dinâmico, o definição de um tipo pode ser alterada em tempo de execução. 

A baixo temos um exemplo da aplicação de monkey patching. Definimos uma classe sem a definição de um método `__len__()`. 

In [6]:
class Colecao():
    def __init__(self):
        self.dados = ['a','b','c']
        
    def __getitem__(self,pos):
        return self.dados[pos]
    
    def __str__(self):
        return "["+ str(self.dados) + "]"
    

Após definir uma classe, criamos uma função separada capaz de imprimir o tamanho do atributo de dados de qualquer argumento passado. 

In [7]:
def obter_len(self):
    return len(self.dados)

Então, atribuímos a referência da função `obter_len()` para o método `__len__()` da classe de Coleção, adicionando um novo método para a classe em tempo de execução.

In [8]:
Colecao.__len__ = obter_len
x = Colecao()
len(x)

3

O monkey patching é possível já que todos métodos recebem o primeiro paramêtro especial chamado comumente de self que representa o objeto. Logo, qualquer função que trata o primeiro paramêtr como refêrencia a um objeto, pode ser incorparado dentro de uma classe. 

É possível utilizar o monkey patching também para definir novos atributos para uma classe. Para definir um valor default, basta atribuir um valor para `Classe.nomeAtributo`. Para alterar valores basta atribuir valores para o atributo usando a referência do objeto.

In [13]:
x.tamanho = len(x)
print(x.tamanho)

3


## Consulta de Atributos
As classes de Python podem ter atributos tantos orientados a instâncias (Ex: Atributo nome, onde cada objeto possuí um valor para nome) ou classes (onde todos os objetos compartilham  o mesmo valor).

In [20]:
class Nova():
    def __init__(self):
        self.valor = 1

Nova.ValorComum = 10

x = Nova()
y = Nova()

x.valor += 1

print(str(x.valor) + " " + str(y.valor))
Nova.ValorComum += 1
print(str(x.ValorComum) + " " + str(y.ValorComum))

2 1
11 11


Note que ao referenciar o atributo da classe como sendo do objeto, o valor do objeto é atualizado usando o valor da classe, entretanto, o valor da classe não se altera. 

In [21]:
x.ValorComum += 1
print(str(x.ValorComum) + " " + str(y.ValorComum))
Nova.ValorComum += 1
print(str(x.ValorComum) + " " + str(y.ValorComum))

12 11
12 12


Para organizar como isto acontece, o Python mantém um dicionário interno para atributos de classe e outro para atributos de objeto. Para acessar cada um basta se acessar o atributo `__dict__`.

In [23]:
print(Nova.__dict__)

{'__module__': '__main__', '__init__': <function Nova.__init__ at 0x000002936418A708>, '__dict__': <attribute '__dict__' of 'Nova' objects>, '__weakref__': <attribute '__weakref__' of 'Nova' objects>, '__doc__': None, 'ValorComum': 12}


In [24]:
print(x.__dict__)

{'valor': 2, 'ValorComum': 12}


Para consultar um atributo de classe, o Python primeiro busca no dicionário pelo atributo. Se um valor não for encontrado, então é buscado no dicionário dos pais da classe. Para consultar um atributo de objeto, o Python busca primeiro no dicionário de instância da classe, se não encontrar, busca no dicionário de classe, e se não encontrar, repete o processo para os dicionários dos pais. 

Como os atributos são armazenados através de dicionários, é possível acessar o atributo e seus valores através do dicionário. Entretanto, acessar o atributo desta forma, não causa o processo de consulta, e o atributo não sera encontrado se ele não for da classe. 

In [25]:
print(x.__dict__['valor'])

2


## Acesso de Atributos desconhecidos
Para lidar com o acesso de atributos não existentes, há outra forma (além do try catch para pegar o AtributteError) para lidar com o problema. Definindo o método `__getattr()__`, que é chamado quando um atributo não é encontrado nos dicionários, é possível realizar o que for necessário para lidar com estes problemas, como gerar uma mensagem de erro ou definir um valor default. 

In [27]:
class Nova():
    def __init__(self):
        self.valor = 1

    def __getattr__(self, attribute):
        print(attribute)
        return 'default'
    
x = Nova()
y = x.avulso
print(y)

avulso
default


Vale comentar qeu a chamada deste método faz parte do processo de consulta de atributos, logo, não é chamado se o acesso for diretamente através do dicionário. 

## Acesso de Métodos Desconhecidos
O método `__getattr__()` também é invocado após métodos que não existem serém chamados. Para lidar com isto, é possível definir um método default na própria classe que é executado quando um método for chamado. Logo, ao invés de retornar um valor default, se retorna uma função default, que é executada já que o método não existente é chamado com `()`. Note quea refêrencia para um método default é retornada quando o valor é um atributo. 

In [28]:
class Nova():
    def __init__(self):
        self.valor = 1

    def __getattr__(self, attribute):
        print(attribute)
        return self.met_def
    
    def met_def(self):
        return 'default'
    
x = Nova()
y = x.avulso
print(y)
x.avulso()

avulso
<bound method Nova.met_def of <__main__.Nova object at 0x00000293640DCAC8>>
avulso


'default'

## Interceptando o processo de consulta de atributo
O processo de consulta de atributo pode ser modificado através do método `__getattribute__()`, que é chamado sempre que se acessa um atributo (e não somente quando ocorre um erro). Este método deve retornar o valor do atributo consultado pelo usuário, ou chamar `__getattr__()` caso o atributo não exista. Esta função é chamada sempre que se usa o `.` para acessar um atributo, logo, é importante não implementar recursividade dentro desta função, tentando acessar um atributo da própria classe usando `.`.  O processo é executado como normalmente seria ao chamar o método `__getattribute__()` da classe object no final do método.

In [30]:
class Nova():
    def __init__(self):
        self.valor = 1
        
    def __getattribute__(self, name):
        print("Consultando " + name + "...")
        # Utilizando processo de consulta de atributo de object
        return object.__getattribute__(self,name)

    def __getattr__(self, attribute):
        print(attribute)
        return self.met_def
    
    def met_def(self):
        return 'default'
    
x = Nova()
print(x.valor)

Consultando valor...
1


## Interceptando o processo de atualização de atributo
Semelhante a forma vista anteriormente, também é possível interceptar o processo de definir valores para atributos através do método `__setattr__()`. Quando a notação de ponto é utilizada para atribuição de valores, este método é chamado. Note que o método é chamado até mesmo pelo método construtor. 

In [6]:
class Nova():
    ops = 0
    
    def __init__(self):
        self.valor = 1
        Nova.ops = 1
        
    def __setattr__(self, atrib, valor):
        print("Atualizando " + str(atrib) + " com valor " + str(valor) + "...")
        return object.__setattr__(self,atrib,valor)
x = Nova()
x.valor = 5

Atualizando valor com valor 1...
Atualizando valor com valor 5...


Note que ao atualizar o valor de ops, o método não é chamado, já que este método só é chamado para atributos do objeto.